In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
# import requests
import time
import random
import os

if not os.path.exists('./restList'):
    os.mkdir('./restList')
    
# if not os.path.exists('./restPhoto'):
#     os.mkdir('./restPhoto')
j=0
url='https://www.tripadvisor.com.tw/Restaurants-g293913-oa{}-Taipei.html'.format(j)

# userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36 Edg/90.0.818.56'
# headers = {'User-Agent':userAgent}

browser=webdriver.Chrome()
t=1
for i in range(150):
    browser.get(url)
    res=browser.page_source
    soup=BeautifulSoup(res, 'html.parser')
    print('********載入第'+str(t)+'頁********')
    for restTags in soup.select('a[class="_15_ydu6b"]'):
        if len(restTags.text.split('. ',1))==1: # 該餐廳為贊助商
            continue
        else:
            restName=restTags.text.split('. ',1)[1]
        try:
            restUrl='https://www.tripadvisor.com.tw'+restTags['href']
            browser.get(restUrl)
            restRes=browser.page_source
            restSoup=BeautifulSoup(restRes, 'html.parser')
            if len(restSoup.select('svg[class="zWXXYhVR"]'))!=0:
                genRating=restSoup.select('svg[class="zWXXYhVR"]')[0]['title'].split(' 分')[0]
            else:
                genRating="None"
            if len(restSoup.select('a[class="_2mn01bsa"]'))!=0 and '$' in restSoup.select('a[class="_2mn01bsa"]')[0].text:
                price=restSoup.select('a[class="_2mn01bsa"]')[0].text
            else:
                price="None"
            if len(restSoup.select('a[class="_15QfMZ2L"]'))!=0:
                ranking=restSoup.select('a[class="_15QfMZ2L"]')[0].text
            else:
                ranking="None"
            if len(restSoup.select('a[class="_15QfMZ2L"]'))>=1:
                location=restSoup.select('a[class="_15QfMZ2L"]')[1].text
            else:
                location="None"
            if len(restSoup.select('span[class="_15QfMZ2L"]'))!=0:
                tel=restSoup.select('span[class="_15QfMZ2L"]')[0].text
            else:
                tel="None"                        
#             website=restSoup.select('a[class="_2wKz--mA _15QfMZ2L"]')[0]['href']
#             imgUrl=restSoup.select('img[class="basicImg"]')[0]['data-lazyurl']
#             resImg = requests.get(imgUrl, headers=headers)
#             resContent = resImg.content
#             with open('./restPhoto/{}.{}'.format(restName.replace('/', '-'), imgUrl.split('.')[-1]), 'wb') as f:
#                 f.write(resContent)
            with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'w', encoding='utf-8') as f:
                f.write('餐廳名稱: '+restName+'\n')
                f.write('餐廳網站: '+restUrl+'\n')
                f.write('區域排名: '+ranking+'\n')
                f.write('餐廳地點: '+location+'\n')
                f.write('餐廳電話: '+tel+'\n')
                f.write('餐廳價位: '+price+'\n')
                f.write('整體評分: '+genRating+'\n')
                
            ratingsList=restSoup.select('div[class="jT_QMHn2"]')
            if len(ratingsList) == 4:
                for ratings in ratingsList:
                    bubbles=ratings.select('span[class="_377onWB-"] span')[0]['class'][1].split('bubble_')[-1]
                    with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                        f.write(ratings.text+': ')
                        f.write(str(int(bubbles)/10)+'\n')
            elif len(ratingsList) == 3:
                for ratings in ratingsList:
                    bubbles=ratings.select('span[class="_377onWB-"] span')[0]['class'][1].split('bubble_')[-1]
                    with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                        f.write(ratings.text+': ')
                        f.write(str(int(bubbles)/10)+'\n')
                with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                    f.write('氛圍: '+'None'+'\n')
            else:
                with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                    f.write('食物: '+'None'+'\n')
                    f.write('服務: '+'None'+'\n')
                    f.write('實惠: '+'None'+'\n')
                    f.write('氛圍: '+'None'+'\n')
                    
            if len(restSoup.select('a[class="_2mn01bsa"]'))!=0 and '$' not in restSoup.select('a[class="_2mn01bsa"]')[0].text:
                restTypeList=restSoup.select('a[class="_2mn01bsa"]')
                with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                    f.write('餐廳類型: ')
                    for restType in restTypeList:
                        f.write(restType.text+'/')
                    f.write(' '+'\n')
            elif len(restSoup.select('a[class="_2mn01bsa"]'))!=0 and '$' in restSoup.select('a[class="_2mn01bsa"]')[0].text: 
                restTypeList=restSoup.select('a[class="_2mn01bsa"]')[1:]
                with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                    f.write('餐廳類型: ')
                    for restType in restTypeList:
                        f.write(restType.text+'/')
                    f.write(' '+'\n')                
            else:
                with open('./restList/{}.txt'.format(restName.replace('/', '-')), 'a', encoding='utf-8') as f:
                    f.write('餐廳類型: '+'None'+'\n')
                    
            print(restName+': 輸出成功')
        
        except IndexError as e:
            print('-----'+restName+': 餐廳資訊問題-----')
            print(e)
        except OSError as e:
            print('-----餐廳命名問題-----')
            print(e)
            
        time.sleep(random.randint(1, 5))
        
    t+=1
    j+=30
#     newUrl = 'https://www.tripadvisor.com.tw'+soup.select('a[class="nav next rndBtn ui_button primary taLnk"]')[0]['href']
    newUrl = 'https://www.tripadvisor.com.tw/Restaurants-g293913-oa{}-Taipei.html'.format(j)
    url=newUrl
            
browser.close()

********載入第1頁********
W XYZ Bar: 輸出成功
艾朋牛排餐酒館: 輸出成功
Le Blanc: 輸出成功
布娜飛比利時啤酒餐廳 台北市民店: 輸出成功
WOOBAR: 輸出成功
布娜飛小酒館 台北中山店: 輸出成功
梅子鰻蒲燒屋日本料理: 輸出成功
The Jade Lounge at Mandarin Oriental Taipei: 輸出成功
The Kitchen Table 西餐廳: 輸出成功
紫艷中餐廳: 輸出成功
紫艷酒吧: 輸出成功
樂軒松阪亭: 輸出成功
香宮: 輸出成功
合.Shabu(合鍋物): 輸出成功
粵亮廣式料理-台北六福萬怡酒店: 輸出成功
胖東西廚房: 輸出成功
地中海牛排館-歐華酒店: 輸出成功
馬友友印度廚房: 輸出成功
Gustoso 義大利餐廳-慕軒飯店: 輸出成功
頤宮中餐廳 Le Palais: 輸出成功
琺蘭綺瑥朵茶餐館: 輸出成功
明宮粵菜廳 台北老爺大酒店: 輸出成功
頁小館: 輸出成功
夢樂小廚: 輸出成功
巴雷巴雷印度餐廳: 輸出成功
樂軒和牛專門店: 輸出成功
梅子餐廰: 輸出成功
馬友友印度廚房（大直店）: 輸出成功
馬友友印度廚房--民生東路: 輸出成功
N°168 Prime 牛排館 維多麗亞酒店: 輸出成功
********載入第2頁********
茹絲葵經典牛排館 民生店: 輸出成功
便所歡樂主題餐廳: 輸出成功
愛新覺羅 原味‧鮮採嚴煮: 輸出成功
鼎泰豐(信義新光A4店): 輸出成功
茹絲葵經典牛排館 大直店: 輸出成功
燈燈庵日式料理餐廳: 輸出成功
原燒 (台北林森北店): 輸出成功
大腕燒肉專門店: 輸出成功
鼎泰豐(101店): 輸出成功
夏慕尼 - 忠孝東路店: 輸出成功
果然匯: 輸出成功
夏慕尼 - 台北中山北店: 輸出成功
Danny & Company: 輸出成功
Prime One牛排館: 輸出成功
鼎泰豐(天母店): 輸出成功
囍聚精緻鍋物: 輸出成功
態芮: 輸出成功
阿城鵝肉: 輸出成功
金鳳廳 - 六福客棧: 輸出成功
文華Cafe: 輸出成功
Rilakkuma CafE: 輸出成功
燒肉同話(台北光復南店): 輸出成功
藝奇 - 台北衡陽店: 輸出成功
Bencotto at Mandarin Oriental, Taipei: 輸出成功

王記府城肉粽(西寧店): 輸出成功
********載入第16頁********
Gēn Creative: 輸出成功
Cheers 台北君悅酒店: 輸出成功
胡同燒肉夜食: 輸出成功
1010湘 - 天母三越店: 輸出成功
DiVino Ristorante Italiano: 輸出成功
老友記粥麵館: 輸出成功
庄頭豆花担: 輸出成功
松江自助火鍋城: 輸出成功
MVSA沐紗西班牙酒莊餐廳: 輸出成功
桔梗三明治: 輸出成功
Din Tai Fung: 輸出成功
HoneyPig 韓式烤肉: 輸出成功
漂亮中式海鮮: 輸出成功
大戶屋 忠孝復興店: 輸出成功
樂麵屋 西門店: 輸出成功
瓦城泰國料理-南港車站店: 輸出成功
Fourplay Cuisine and Bar: 輸出成功
秦小姐豆漿店: 輸出成功
鬍鬚張魯肉飯美食文化館 - 寧夏路: 輸出成功
馬友友印度廚房 - 松江路: 輸出成功
紐西蘭風味漢堡: 輸出成功
TGI Fridays 星期五餐廳 忠孝店: 輸出成功
犇 和牛館 牛排 下午茶: 輸出成功
Bing: 輸出成功
老牌張豬腳飯: 輸出成功
正好鮮肉小籠湯包: 輸出成功
三朵花炭火燒肉: 輸出成功
鬍子餐酒: 輸出成功
饗食天堂大直店: 輸出成功
清真泰富豪: 輸出成功
********載入第17頁********
九寨十鍋正宗四川麻辣鍋: 輸出成功
Urban 331-慕軒飯店: 輸出成功
欣葉台菜-忠孝店: 輸出成功
蘇阿姨比薩屋: 輸出成功
SONOMA Global Steak House 焱牛排館: 輸出成功
野草居食屋: 輸出成功
穗科手打烏龍麵: 輸出成功
詹記麻辣火鍋(南京店): 輸出成功
水龜伯古早味: 輸出成功
Wired Chaya 茶屋 (內湖): 輸出成功
台北威斯汀六福皇宮 - 祇園日本料理: 輸出成功
瓦城泰國料理 - 美麗華店: 輸出成功
Restaurant Ephernite: 輸出成功
Ed's Diner: 輸出成功
小蒙牛頂級麻辣養生鍋 公館店: 輸出成功
紫琳蒸餃館: 輸出成功
天喜迷你火鍋: 輸出成功
兔卡蕾餐酒館: 輸出成功
Mia Cucina: 輸出成功
継光香香鶏: 輸出成功
饗聚廚房: 輸出成功
EPL Steakhouse & Lounge: 輸出成功
番紅花城土耳其餐廳: 輸出成功

丸林魯肉飯: 輸出成功
Jimolulu-A Taste of Hawaii Paradise 美式夏威夷: 輸出成功
游壽司: 輸出成功
********載入第32頁********
疊疊樂 松菸分店: 輸出成功
蝸牛義大利餐廳-敦南店: 輸出成功
黒岩 黒砂糖剉冰: 輸出成功
阿宗麵線忠孝店: 輸出成功
極野宴燒肉專門店 (台北sogo旗艦店): 輸出成功
金龍餐廳: 輸出成功
肥猫漁夫: 輸出成功
徐州路2號庭園會館: 輸出成功
台北永和豆漿大王: 輸出成功
明德素食園（京站店）: 輸出成功
大稻埕魯肉飯: 輸出成功
LeTAO小樽洋菓子舖(松菸店): 輸出成功
巴獅子德國餐廳: 輸出成功
小龍飲食店: 輸出成功
Am Daily: 輸出成功
石二鍋 - 台北信義店: 輸出成功
無老鍋: 輸出成功
饗樂精緻鍋品: 輸出成功
新馬辣經典麻辣鍋 - 漢中店: 輸出成功
小王清湯瓜仔肉: 輸出成功
涓豆腐: 輸出成功
21街冰棧: 輸出成功
海峽會餐飲會所: 輸出成功
山海樓: 輸出成功
Taipei 47名人宴(台北47): 輸出成功
皮丘咖啡: 輸出成功
薄多義(市府店): 輸出成功
窩台北: 輸出成功
VG Cafe' Taipei: 輸出成功
黑毛屋 信義店: 輸出成功
********載入第33頁********
Ruby Cafe 義法料理: 輸出成功
Khana khazana Indian kitchen: 輸出成功
雅朵義大利餐館: 輸出成功
HYG: 輸出成功
段純貞牛肉麵: 輸出成功
金色三麥 - 信義誠品店: 輸出成功
建宏牛肉麵: 輸出成功
小南門豆花: 輸出成功
苦茶之家: 輸出成功
辣麻味噌拉麵 鬼金棒: 輸出成功
海真私房菜: 輸出成功
高記復興店: 輸出成功
植宿: 輸出成功
MissGreen: 輸出成功
又一村水餃麵食館: 輸出成功
21風味館: 輸出成功
意麵王: 輸出成功
溫德德式烘培餐館 - 大安店: 輸出成功
微風建一食堂: 輸出成功
雞翅啤酒吧: 輸出成功
四知堂: 輸出成功
宋廚北方菜館: 輸出成功
大戶屋: 輸出成功
加賀伊: 輸出成功
老爺小花園: 輸出成功
Sunny Buffet: 輸出成功
賴記雞蛋蚵仔煎: 輸出成功
南街得意茶館: 輸出成功
福大山蒸餃館: 輸出成功
饗泰多 - 松高店: 輸出成功
***

Popina: 輸出成功
香料屋印度料理: 輸出成功
WINONA 葡萄酒餐廳: 輸出成功
別所: 輸出成功
Trio cafe: 輸出成功
旅人咖啡: 輸出成功
Kaya Kaya Café: 輸出成功
牛棒碗安 GOBO 精緻鍋物: 輸出成功
燈籠滷味: 輸出成功
宴客樓(台北萬豪酒店): 輸出成功
喀佈貍-大眾和風洋食居酒屋: 輸出成功
甘記華茄刀削麺: 輸出成功
世代歐陸牛排西餐廳: 輸出成功
OZ Cafe & Bistro: 輸出成功
特有種商行: 輸出成功
********載入第49頁********
松園禪林餐廳: 輸出成功
朝鮮味韓國料理: 輸出成功
ABV 美式餐酒館: 輸出成功
Cafe Solo Singer Life: 輸出成功
小南門: 輸出成功
Mad for Garlic: 輸出成功
海天香餃: 輸出成功
子辣 個人 麻辣: 輸出成功
春梅子餐廳: 輸出成功
角公園咖啡: 輸出成功
すき家（民權西路店）: 輸出成功
問鼎麻辣養生鍋 - 西門: 輸出成功
一頭也黑毛和牛燒肉店: 輸出成功
邊田庄松江旗艦會館: 輸出成功
荷蘭小鬆餅南西中山店: 輸出成功
沐蔬輕食尚: 輸出成功
好小子海鮮店: 輸出成功
吳家牛肉麵: 輸出成功
Cafe52: 輸出成功
大方冰品: 輸出成功
老蔡水煎包: 輸出成功
長春素食餐廳: 輸出成功
東輝韓食館: 輸出成功
Miacucina (My kitchen) 復興店: 輸出成功
Zui Ma la: 輸出成功
小紐約披薩: 輸出成功
Pin Chuan Lan: 輸出成功
老蔡水煎包: 輸出成功
江雁塘新時尚鐵板燒: 輸出成功
下港臭豆腐: 輸出成功
********載入第50頁********
方家雞肉飯: 輸出成功
Little London Taipei: 輸出成功
江南春 - 台北福華大飯店: 輸出成功
牛壽司: 輸出成功
CASA DELLA PASTA義麵坊 中山店: 輸出成功
湘鼎坊: 輸出成功
老张牛肉面店: 輸出成功
大三元酒樓: 輸出成功
点爭鮮: 輸出成功
現烘咖啡專門店忠孝杭州店: 輸出成功
開機咖啡: 輸出成功
永和豆漿: 輸出成功
五湖豆漿: 輸出成功
躼腳: 輸出成功
The Pita Bar: 輸出成功
大戶屋 - 微風廣場店: 輸出成功
1010湘-西湖店: 輸出成功
H

金子半之助 - 新光三越信義A8店: 輸出成功
六品中式料理餐廳（内湖店）: 輸出成功
蔡記隆府 龍頭寺老灶火鍋: 輸出成功
吉園葡花園野菜餐廳: 輸出成功
風食堂: 輸出成功
旺角迷你石頭火鍋(中華店): 輸出成功
良粟商號: 輸出成功
宇宙人主題餐廳: 輸出成功
清心福全 昆明店: 輸出成功
Wildwood Live Fire Cuisine: 輸出成功
開丼: 輸出成功
銀杏川酒菜館: 輸出成功
大户屋-統一時代台北店: 輸出成功
初心菓寮: 輸出成功
天星港式飲茶: 輸出成功
周胖子餃子館 - 忠孝店: 輸出成功
陳陽鵝肉大王: 輸出成功
加福奇士蛋糕專門店: 輸出成功
Clear-Cut: 輸出成功
品冠客家小館: 輸出成功
一風堂微風台北車站店: 輸出成功
北村家小料理屋: 輸出成功
八方雲集: 輸出成功
北平稻香村: 輸出成功
新葡苑 新光三越信義新天地A9館: 輸出成功
鳥家燒烤: 輸出成功
定食8 - 林森南路: 輸出成功
********載入第66頁********
定食8 - 林森南路: 輸出成功
御鼎香脆皮烤鴨餐廳: 輸出成功
宜品福州乾拌麵: 輸出成功
口吅品麻辣臭豆腐 士林店: 輸出成功
The Three Lions Inn: 輸出成功
小巷亭: 輸出成功
御盤De食堂 (微風南京店): 輸出成功
麥當勞: 輸出成功
戲台茶館: 輸出成功
貳樓餐廳 - 師大店: 輸出成功
和幸沖繩居酒屋: 輸出成功
犇鐵板燒: 輸出成功
佐藤精肉店: 輸出成功
佐藤咖哩: 輸出成功
SUBWAY 台北101店: 輸出成功
黑岩古早味黑砂糖剉冰錦州店: 輸出成功
馬六甲馬來西亞風味館 - 安和店: 輸出成功
台北泰 旗艦店: 輸出成功
老外一品牛肉麵: 輸出成功
易牙居點心坊(羅斯福路二店): 輸出成功
和民之家 - 站前店: 輸出成功
佳香點心大王: 輸出成功
自然食 - 科技大樓: 輸出成功
Lu Seafood Steam Pot: 輸出成功
合掌村: 輸出成功
365 Xiao Chi Dian - Little Eat Place: 輸出成功
東北軒酸菜白肉鍋: 輸出成功
景美鄭家碳烤: 輸出成功
鄭記豬腳飯: 輸出成功
金爸爸 南港店: 輸出成功
********載入第67頁********
鳥哲 燒物專門店: 輸出成功
雞家莊-六條店: 

阿財鍋貼水餃專賣店: 輸出成功
匠饗fusion居酒屋: 輸出成功
璞膳日式鍋物: 輸出成功
燒肉眾: 輸出成功
小農鍋物 大安店: 輸出成功
穗浪寿司和食專門店: 輸出成功
三角埔仙草: 輸出成功
冶春茶社: 輸出成功
漢堡王 林森店: 輸出成功
光點華山 珈琲時光: 輸出成功
VVg Thinking: 輸出成功
子元壽司 割烹: 輸出成功
山形心心拉麵: 輸出成功
貓空 六季香茶坊: 輸出成功
元氣貓主題餐廳: 輸出成功
********載入第82頁********
YABI Kitchen-微風南山店: 輸出成功
誠記越南麵食館 - 永吉店: 輸出成功
義式屋古拉爵: 輸出成功
麥村: 輸出成功
周胖子餃子館: 輸出成功
十十燒烤苑: 輸出成功
申浦尚宴: 輸出成功
泰酷.泰國料理: 輸出成功
Plan B 欧陸街頭市集小酒館: 輸出成功
度小月 - 迪化店: 輸出成功
鈜景國產肉品專賣店東門店 (御牛殿) - 台北東門店: 輸出成功
月湖歐式料理: 輸出成功
江浙秀蘭小館: 輸出成功
頑古養生牛肉麵: 輸出成功
392 Cafe: 輸出成功
好吃羊肉爐: 輸出成功
魚兵衛太春和食處: 輸出成功
穗科食堂: 輸出成功
涮鍋里: 輸出成功
雪嶽山韓式料理: 輸出成功
怡園中餐廳: 輸出成功
茉莉漢堡: 輸出成功
忠南飯館: 輸出成功
阿國切仔麵: 輸出成功
旗魚新竹米粉: 輸出成功
時殺互娛服份有限公司: 輸出成功
SKYLARK加州風洋食館: 輸出成功
La Mole Taipei: 輸出成功
農人餐桌親子餐廳: 輸出成功
松滿樓: 輸出成功
********載入第83頁********
木待制咖啡: 輸出成功
利隆餅店: 輸出成功
大眾酒場十五番地: 輸出成功
東門鴨莊: 輸出成功
鍋膳新京都: 輸出成功
Choir Cafe: 輸出成功
時時爐端燒: 輸出成功
Frank Taipei: 輸出成功
5 Senses Cafe: 輸出成功
金山客家小館 光復店: 輸出成功
Zoom In: 輸出成功
芝玫蛋糕: 輸出成功
江夏御品冠軍牛肉麵: 輸出成功
築地鮮魚重慶店: 輸出成功
有沒有咖啡 壹號店: 輸出成功
財神台南擔仔麵: 輸出成功
亞廬意大利窯烤: 輸出成功
南大門韓國烤肉專賣店: 輸出成功
picnic野餐咖啡: 輸出成功
極鮮流精緻海鮮涮涮鍋:

小後苑: 輸出成功
莎諾歐法西餐: 輸出成功
于記杏仁豆腐 通化店: 輸出成功
德意事歐式美食屋: 輸出成功
Sukiya（信義安和店）: 輸出成功
蔬之膳義法蔬食餐廳: 輸出成功
金澤日式居酒屋: 輸出成功
佳興福州魚丸店: 輸出成功
Perfume Dance 跳舞香水: 輸出成功
真芳-碳烤吐司 (民生店): 輸出成功
Bellini Pasta Pasta貝里尼(ATT忠孝店): 輸出成功
一村水餃麵食館: 輸出成功
可味包子饅頭專賣店: 輸出成功
師園鹽酥雞 - 西門店: 輸出成功
Bonjour 朋廚: 輸出成功
大龍蝦海鮮餐廳: 輸出成功
西堤 - 台北復興南店: 輸出成功
Season Cuisine Patissiartism: 輸出成功
阿里郎韓國小吃: 輸出成功
愛烤愛對囉 - 士林店: 輸出成功
八方雲集(承德店): 輸出成功
豐盛號: 輸出成功
上樓看看 Arthere Cafe: 輸出成功
高麗棒韓國烤肉店: 輸出成功
天仁茗茶 - 大安店: 輸出成功
桂丁香印度料理: 輸出成功
ROYAL HOST樂雅樂家庭餐廳(北投店): 輸出成功
紅蜻蜓食事處: 輸出成功
澳氏咖啡: 輸出成功
台南三哥海産店: 輸出成功
********載入第100頁********
Tao Ban Woo: 輸出成功
烤師傅 永吉店: 輸出成功
櫻桃小鎮: 輸出成功
呷七碗: 輸出成功
乾杯黑毛屋: 輸出成功
山豬蛋餅: 輸出成功
富順樓餐廳: 輸出成功
涓豆腐: 輸出成功
天和烘培: 輸出成功
肌肉海灘 - 南京概念餐廳: 輸出成功
馬祖麵館: 輸出成功
阿唐燒餅舖: 輸出成功
王家水餃館: 輸出成功
彭園婚宴會館: 輸出成功
黑木屋: 輸出成功
阿輝伯蘿蔔絲餅: 輸出成功
早安.勇士 自家烘培咖啡館: 輸出成功
川越駅日式串燒專門店: 輸出成功
Teotihuacán 特奧蒂瓦坎 墨西哥料理: 輸出成功
花甜果室: 輸出成功
首都-公館店: 輸出成功
秋料理精緻和食: 輸出成功
晶湯匙泰式主題餐廳 天母高島屋店: 輸出成功
Commons: 輸出成功
鬍鬚張(台北南門店): 輸出成功
補時咖啡: 輸出成功
千暉鵝肉: 輸出成功
合點壽司 -內湖店: 輸出成功
林葱抓餅: 輸出成功
巷上食璞川菜客家菜: 輸出成功
********載入第101頁********

松發肉骨茶(遠百信義A13): 輸出成功
大戶屋 (遠百寶慶店): 輸出成功
加爾各答 印度咖哩屋: 輸出成功
安和担仔麵: 輸出成功
拓樸本然Topo Cafe: 輸出成功
喜揚揚揚州灌湯包: 輸出成功
Fujiflower cafe - 松菸店: 輸出成功
醬太郎日式燒肉: 輸出成功
小滿苑燒肉: 輸出成功
運鈍根湯(安和店): 輸出成功
海壽司: 輸出成功
串炸達摩 串炸: 輸出成功
Angle Cafe': 輸出成功
磚窯古早料理: 輸出成功
雲泰風味南國料理: 輸出成功
清真泰皇餐廳: 輸出成功
柳家涼麵: 輸出成功
大戶屋 (松山車站店): 輸出成功
Shichido Sushi New Food Japanese Food: 輸出成功
天命庵-日式串燒、熟食: 輸出成功
辣霸子麻辣鴛鴦火鍋: 輸出成功
山边咖啡: 輸出成功
1885 Burger Store - 南京店: 輸出成功
新帆船澳門葡國餐廳: 輸出成功
慢動作咖啡館: 輸出成功
Yong Kang Beef Noodle: 輸出成功
********載入第117頁********
勝口味麵線: 輸出成功
Revel Eatery and Juice Bar: 輸出成功
山東鹿家莊麵食館: 輸出成功
火鍋106-粵式豬肚煲鍋: 輸出成功
麥當勞 (內湖): 輸出成功
Inn Cafe 成都薆悅精品酒店: 輸出成功
廚房客家美食: 輸出成功
At. First Brunch: 輸出成功
Go Greek Restaurant: 輸出成功
Chili's: 輸出成功
大村武串燒居酒屋: 輸出成功
花園小酒館: 輸出成功
若獅子 和食 酒处: 輸出成功
歐麗蛋糕坊: 輸出成功
小曼谷滇泰料理: 輸出成功
大阪燒肉雙子: 輸出成功
顏記杏仁露: 輸出成功
阿宏烏骨雞: 輸出成功
心齋: 輸出成功
上官木桶鍋 - 中正店: 輸出成功
良芳餐館: 輸出成功
角落咖啡: 輸出成功
We Love Cookies: 輸出成功
兔子兔子 美式漢堡: 輸出成功
上台北小館: 輸出成功
一之軒 - 南東旗艦店: 輸出成功
Imp Cafe: 輸出成功
永康冰鎮: 輸出成功
瘋麻辣頂級鴛鴦火鍋: 輸出成功
Aluvbe Cakery Taipei: 輸出成功
********載入第118頁********
Aluvb

鍋無敵: 輸出成功
石家割包(總店): 輸出成功
Shao Shao Ke Shang-Hsi Cousine/: 輸出成功
必勝客 士林中正店: 輸出成功
伯朗咖啡館: 輸出成功
好好食房 Soup Up: 輸出成功
喫茶趣 (信義店): 輸出成功
北北车鲁肉饭: 輸出成功
天母盛鑫 - 麗華館 (美麗華百樂園): 輸出成功
上官木桶锅 乐利店: 輸出成功
阿力的搖滾廚房: 輸出成功
168排骨飯: 輸出成功
Siris義法料理餐廳: 輸出成功
Taiwanippon: 輸出成功
雙人徐: 輸出成功
騾老么牛肉麵: 輸出成功
Moe Yakiniku: 輸出成功
阿英海產粥: 輸出成功
意麵王 士林店: 輸出成功
********載入第134頁********
意麵王 士林店: 輸出成功
柳家涼麵: 輸出成功
福園簡餐: 輸出成功
Power Beef: 輸出成功
好奇豆花: 輸出成功
愛天空: 輸出成功
雙喜: 輸出成功
一条通 - 石牌門市: 輸出成功
京町八號: 輸出成功
RD Cafe 深烘焙(重慶南店): 輸出成功
La Farfalla 義式餐廳 - 寒舍艾麗酒店: 輸出成功
仙客來海鮮庭園餐廳: 輸出成功
禧太炭火燒肉: 輸出成功
黑鳥先生咖啡廳: 輸出成功
大師兄銷魂麵舖-信義店: 輸出成功
彭記蚵仔麵脆皮臭豆腐: 輸出成功
旺．慢食餐酒館 - 中山店: 輸出成功
La Pizza 石窯風餐館: 輸出成功
Lecker里克德義廚房: 輸出成功
Season Artisan Pâtissier(敦南店): 輸出成功
新北泰雲泰特色料理: 輸出成功
Captain Lobster: 輸出成功
開丼: 輸出成功
東港金婆飯湯: 輸出成功
Winnie's European Cuisine: 輸出成功
碧耳貓: 輸出成功
一郎精緻日式料理(中正店): 輸出成功
鉄火牛排 - 新光三越: 輸出成功
長榮桂冠酒店La Petite Cuisine: 輸出成功
瑞安豆漿大王: 輸出成功
********載入第135頁********
Doggy & Yummy: 輸出成功
越苑美式越南河粉(敦北店): 輸出成功
大塊牛排專賣店: 輸出成功
CAMPUS CAFE 光復店: 輸出成功
伊通街知高飯: 輸出成功
鯊魚咬吐司: 輸出成功
龍茶味點: 輸出成功
山丼: 

齊味餃子館: 輸出成功
食事処: 輸出成功
芳芳江蘇水餃: 輸出成功
圓滿川湘菜海鮮: 輸出成功
加福奇士蛋糕專門店: 輸出成功
西湖水產: 輸出成功
旅東京鐵板小料理: 輸出成功
夏圓紅豆餅: 輸出成功
山東姥姥麵食: 輸出成功
Komeda‘s Coffee - Xiao Ju Dan: 輸出成功
瀟湘園私家菜館: 輸出成功
Tendon Tenya -  Breeze NanShan: 輸出成功
An Burger: 輸出成功
馬路口烘焙小舖: 輸出成功
Monga Chicken: 輸出成功
竹葉日本家庭料理: 輸出成功
禾記港式麻辣臭豆腐: 輸出成功
小珍園拉麵: 輸出成功
肯德基  大直愛買: 輸出成功
麵食主義: 輸出成功
菓風小舖: 輸出成功
中島水產(Sogo忠孝館): 輸出成功
真芳: 輸出成功
元香沙茶火鍋: 輸出成功
CoCo壱番屋 誠品站前Express店: 輸出成功
西門老媽蒸餃: 輸出成功
洋城義大利餐廳: 輸出成功


In [2]:
browser.close()

In [1]:
import pandas as pd
import glob

data = list()
for filePath in glob.glob('./restList/*.txt'):
    with open(filePath, 'r', encoding='utf-8') as f:
        tmpList = f.read().split('\n')
        data.append(tmpList)
columns = [r.split(': ') [0] for r in data[0]]

df = pd.DataFrame(data=data, columns=columns)

for c in df:
    df[c] = df[c].apply(lambda x: x.split(': ')[-1])
# df

,餐廳名稱,餐廳網站,區域排名,餐廳地點,餐廳電話,餐廳價位,整體評分,食物,服務,實惠,氛圍,餐廳類型,
0,0315串燒,https://www.tripadvisor.com.tw/Restaurant_Revi...,,台灣 松山台北 松山区松河街214号,None,None,0 則評論,None,None,None,None,中式料理/,
1,1+1 Together Republic,https://www.tripadvisor.com.tw/Restaurant_Revi...,"中山區排名第 #1,555 的餐廳，共 1,653 名","104 台灣 中山區台北 No. 10-1, Lane 26, Section 2, Zho...",+886 2 2568 1997,None,3.0,None,None,None,None,None,
2,1+1創意廚房,https://www.tripadvisor.com.tw/Restaurant_Revi...,松山排名第 #769 的餐廳，共 922 名,105 台灣 松山台北 南京东路五段66巷2弄5号,+886 2 2767 4522,None,4.0,None,None,None,None,義式料理/,
3,1010湘 - 天母三越店,https://www.tripadvisor.com.tw/Restaurant_Revi...,士林排名第 #15 的餐廳，共 669 名,111 台灣 士林台北 天母东路68号 新光三越天母店A栋7楼,+886 2 2875 2955,$$ - $$$,4.5,4.5,4.5,4.0,4.5,泰式料理/,
4,1010湘-信義誠品店,https://www.tripadvisor.com.tw/Restaurant_Revi...,信義區排名第 #23 的餐廳，共 936 名,110 台灣 信義區台北 松高路11号 6f,+886 2 2722 0583,$$ - $$$,4.5,4.5,4.5,4.0,4.0,中式料理/提供純素選擇/,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11599,龐德羅莎,https://www.tripadvisor.com.tw/Restaurant_Revi...,內湖排名第 #141 的餐廳，共 412 名,"114 台灣 內湖台北 No. 68, Xinhu 2nd Rd., Neihu Dist 2f",+886 2 2793 2493,$$ - $$$,3.5,4.0,3.5,4.0,None,美式料理/牛排/,
11600,龐德羅莎牛沙拉（京華城店）,https://www.tripadvisor.com.tw/Restaurant_Revi...,松山排名第 #893 的餐廳，共 922 名,"10562 台灣 松山台北 八德路四段138号 京华城购物中心,B3",+886 2 3762 1998,None,3.0,None,None,None,None,美式料理/牛排/,
11601,龔媽早餐吧,https://www.tripadvisor.com.tw/Restaurant_Revi...,,"台灣 北投台北 No. 273, Sec. 1, Xian St., Beitou Dist",+886 2 2822 8404,None,0 則評論,None,None,None,None,亞洲料理/台灣小吃/台菜/,
11602,龜山島海產店,https://www.tripadvisor.com.tw/Restaurant_Revi...,None,None,None,None,None,None,None,None,None,None,


In [2]:
df.to_csv('./restList.csv', index=False, encoding='utf-8-sig')